### SQL запросы 
В рамках данного ноутбука будем писать запросы к нашей БД 

to do описать в чем удобство использования пандас для выполнения запросов и sqlite (сравнить
)

In [37]:
import sqlite3
import pandas as pd

from sqlalchemy import create_engine

# путь к БД
DB_PATH = "bank.db"

In [33]:
# выполним запрос, используя pandas
engine = create_engine("sqlite:///bank.db")

# топ-5 клиентов по доходу
query = """
SELECT
    id AS client_id,
    fullname,
    income
FROM 
    clients
ORDER BY 
    income DESC
LIMIT 5;
"""
top_income_clients = pd.read_sql(query, engine)
top_income_clients

,client_id,fullname,income
0,752,Никонов Сидор Вячеславович,299768.0
1,577,София Петровна Соловьева,297761.0
2,850,Мамонтова Кира Филипповна,297719.0
3,615,Панова Раиса Леонидовна,296916.0
4,256,Глеб Демидович Лукин,296415.0


In [23]:
# количество клиентов и средний доход по полу
query = """
SELECT
    gender,
    COUNT(*) AS client_count,
    ROUND(AVG(income), 2) AS avg_income
FROM 
    clients
GROUP BY 
    gender;
"""
gender_stats = pd.read_sql(query, engine)
gender_stats

,gender,client_count,avg_income
0,F,475,201882.68
1,M,525,196664.64


In [35]:
# топ-5 клиентов по суммарным расходам (Negative-транзакции)
query = """
SELECT
    t.client_id,
    SUM(
        CASE 
            WHEN t.transaction_type = 'Negative' THEN t.amount 
        ELSE 0 END
    ) AS total_spend
FROM 
    transactions t
GROUP BY 
    t.client_id
ORDER BY 
    total_spend DESC
LIMIT 5;
"""
top_spenders = pd.read_sql(query, engine)
top_spenders

,client_id,total_spend
0,39,1585812
1,295,1536043
2,383,1458210
3,954,1447851
4,309,1429278


In [36]:
# категории, по которым траты выше среднего по всем категориям (подзапрос)
query = """
WITH cat_spend AS (
    SELECT
        product_category,
        SUM(CASE WHEN transaction_type = 'Negative' THEN amount ELSE 0 END) AS spend
    FROM 
        transactions
    GROUP BY 
        product_category
)
SELECT
    c.id AS category_id,
    c.name AS category_name,
    cs.spend
FROM 
    cat_spend cs
JOIN categories c ON c.id = cs.product_category
WHERE 
    cs.spend > (
        SELECT AVG(spend) FROM cat_spend
    )
ORDER BY 
    cs.spend DESC;
"""
above_avg_categories = pd.read_sql(query, engine)
above_avg_categories

,category_id,category_name,spend
0,17,Дом и ремонт,254694724
1,13,Одежда и обувь,131786476
2,2,Супермаркеты,121186954
3,28,Переводы,68489232
4,5,Фастфуд,48740746
5,14,Рестораны,42154724
6,12,Образование,39591574
7,11,Красота,36040369


### Месячные расходы клиентов и их доходы

Считаем месячные расходы клиентов и их долю от дохода, чтобы быстро оценить финансовую нагрузку и выявить потенциальные зоны риска.  
На выходе получаем агрегированную таблицу сlient-month с ключевыми метриками: monthly_spend, income, spend_ratio и простым risk_flag, которая станет базой для дальнейших оконных расчётов, анализа подписок и сегментации клиентов.

In [42]:
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.executescript("""
DROP VIEW IF EXISTS v_monthly_base;

CREATE VIEW v_monthly_base AS
    WITH txn_monthly AS (
        SELECT
            client_id,
            strftime('%Y-%m', date) AS ym,
            SUM(CASE WHEN transaction_type = 'Negative' THEN amount ELSE 0 END) AS monthly_spend,
            SUM(CASE WHEN transaction_type != 'Negative' THEN amount ELSE 0 END) AS monthly_inflow
        FROM 
            transactions
        GROUP BY 
            client_id, ym
    )"""
)
conn.commit()

cur.execute("""
    SELECT
    t.client_id,
    t.ym,
    t.monthly_spend,
    t.monthly_inflow,
    c.income,
    c.expenses,
    ROUND(1.0 * t.monthly_spend / NULLIF(c.income, 0), 3) AS spend_ratio,
    CASE
        WHEN c.income IS NULL OR c.income = 0 THEN 'unknown'
        WHEN 1.0 * t.monthly_spend / c.income > 1.0 THEN 'over_spending'
        WHEN 1.0 * t.monthly_spend / c.income > 0.6 THEN 'risk'
        WHEN 1.0 * t.monthly_spend / c.income > 0.3 THEN 'attention'
        ELSE 'ok'
    END AS risk_flag
FROM 
    txn_monthly t
JOIN clients c ON c.id = t.client_id
LIMIT 5;
""")

rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()

OperationalError: incomplete input

In [45]:
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.executescript("""
DROP VIEW IF EXISTS v_monthly_base;

CREATE VIEW v_monthly_base AS
WITH txn_monthly AS (
    SELECT
        client_id,
        strftime('%Y-%m', date) AS ym,
        SUM(CASE WHEN transaction_type = 'Negative' THEN amount ELSE 0 END) AS monthly_spend,
        SUM(CASE WHEN transaction_type != 'Negative' THEN amount ELSE 0 END) AS monthly_inflow
    FROM transactions
    GROUP BY client_id, ym
)
SELECT
    t.client_id,
    t.ym,
    t.monthly_spend,
    t.monthly_inflow,
    c.income,
    c.expenses,
    ROUND(1.0 * t.monthly_spend / NULLIF(c.income, 0), 3) AS spend_ratio,
    CASE
        WHEN c.income IS NULL THEN 'unknown'
        WHEN c.income = 0 THEN 'unknown'
        WHEN 1.0 * t.monthly_spend / c.income > 1.0 THEN 'over_spending'
        WHEN 1.0 * t.monthly_spend / c.income > 0.6 THEN 'risk'
        WHEN 1.0 * t.monthly_spend / c.income > 0.3 THEN 'attention'
        ELSE 'ok'
    END AS risk_flag
FROM txn_monthly t
JOIN clients c ON c.id = t.client_id
ORDER BY t.client_id, t.ym;
""")

conn.commit()
conn.close()

print("✅ VIEW v_monthly_base успешно создано")

# теперь вывод
conn = sqlite3.connect(DB_PATH)

df = pd.read_sql("""
    SELECT *
    FROM v_monthly_base
    LIMIT 10;
""", conn)

conn.close()

df

✅ VIEW v_monthly_base успешно создано


,client_id,ym,monthly_spend,monthly_inflow,income,expenses,spend_ratio,risk_flag
0,1,2020-01,52842,3125,230910.0,61975,0.229,ok
1,1,2020-02,43243,3473,230910.0,61975,0.187,ok
2,1,2020-03,48511,3528,230910.0,61975,0.210,ok
3,1,2020-04,57227,11606,230910.0,61975,0.248,ok
4,1,2020-05,107511,6555,230910.0,61975,0.466,attention
5,1,2020-06,52545,5256,230910.0,61975,0.228,ok
6,1,2020-07,51841,5368,230910.0,61975,0.225,ok
7,1,2020-08,50787,11315,230910.0,61975,0.220,ok
8,1,2020-09,53659,4179,230910.0,61975,0.232,ok
9,1,2020-10,57769,8941,230910.0,61975,0.250,ok


In [13]:
# распределение флагов
monthly_df["risk_flag"].value_counts()

risk_flag
unknown          7632
attention        2099
ok               1332
risk              735
over_spending     202
Name: count, dtype: int64

Большая часть клиент-месяцев попала в категорию unknown — это связано с отсутствием или нулевым значением дохода в профильных данных.  
Около четверти периодов находятся в зоне attention — умеренная нагрузка расходов.  
Клиентов с высоким уровнем финансового стресса (risk и over_spending) сравнительно немного, но они представляют наибольший интерес для дальнейшего анализа: их расходы систематически превышают 60% доходов, а иногда даже полностью перекрывают их.  

Такая структура распределения показывает, что:
- профильные данные по доходам неполные или частично отсутствуют  
- значительная часть клиентов тратит в пределах комфортного диапазона  
- существует небольшая, но важная группа клиентов с потенциальными финансовыми трудностями

In [49]:
query = """
SELECT
    client_id,
    ym,
    monthly_spend,
    monthly_inflow,
    income,
    expenses,
    spend_ratio,
    risk_flag,

    --  накопительные расходы клиента
    SUM(monthly_spend) OVER (
        PARTITION BY client_id        
        ORDER BY ym                   
    ) AS cumulative_spend,

    -- скользящее среднее расходов за 3 месяца
    ROUND(AVG(monthly_spend) OVER (
        PARTITION BY client_id
        ORDER BY ym
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ), 2) AS avg_3m_spend,

    -- расходы в предыдущем месяце
    LAG(monthly_spend, 1) OVER (
        PARTITION BY client_id
        ORDER BY ym
    ) AS prev_month_spend

FROM 
    v_monthly_base
ORDER BY 
    client_id, ym
LIMIT 5;
"""

window_df = pd.read_sql(query, engine)
window_df


,client_id,ym,monthly_spend,monthly_inflow,income,expenses,spend_ratio,risk_flag,cumulative_spend,avg_3m_spend,prev_month_spend
0,1,2020-01,52842,3125,230910.0,61975,0.229,ok,52842,52842.00,NaN
1,1,2020-02,43243,3473,230910.0,61975,0.187,ok,96085,48042.50,52842.0
2,1,2020-03,48511,3528,230910.0,61975,0.210,ok,144596,48198.67,43243.0
3,1,2020-04,57227,11606,230910.0,61975,0.248,ok,201823,49660.33,48511.0
4,1,2020-05,107511,6555,230910.0,61975,0.466,attention,309334,71083.00,57227.0


In [18]:
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.executescript("""
DROP VIEW IF EXISTS v_client_category_spend;

CREATE VIEW v_client_category_spend AS
WITH tx_cat AS (
    SELECT
        t.client_id,
        t.product_category,
        -- суммарные расходы клиента по каждой категории
        SUM(CASE WHEN t.transaction_type = 'Negative' THEN t.amount ELSE 0 END) AS spend
    FROM transactions t
    GROUP BY t.client_id, t.product_category
)
SELECT
    tc.client_id,
    tc.product_category,
    c.name AS category_name,
    tc.spend,

    -- общие расходы клиента по всем категориям
    SUM(tc.spend) OVER (
        PARTITION BY tc.client_id
    ) AS total_spend_client,

    -- доля категории в общих расходах клиента
    ROUND(
        1.0 * tc.spend
        / NULLIF(
            SUM(tc.spend) OVER (
                PARTITION BY tc.client_id
            ), 0
        ),
        3
    ) AS category_share,

    -- ранг категории по сумме трат (1 = самая большая категория для клиента)
    RANK() OVER (
        PARTITION BY tc.client_id
        ORDER BY tc.spend DESC
    ) AS category_rank

FROM 
    tx_cat tc
LEFT JOIN categories c ON c.id = tc.product_category;
""")

conn.commit()
conn.close()

In [19]:
conn = sqlite3.connect(DB_PATH)

cat_df = pd.read_sql("""
    SELECT *
    FROM v_client_category_spend
    ORDER BY client_id, category_rank;
""", conn)

conn.close()

cat_df.head(15)

,client_id,product_category,category_name,spend,total_spend_client,category_share,category_rank
0,1,13,Одежда и обувь,130068,743707,0.175,1
1,1,2,Супермаркеты,124999,743707,0.168,2
2,1,17,Дом и ремонт,122239,743707,0.164,3
3,1,28,Переводы,96992,743707,0.130,4
4,1,5,Фастфуд,57310,743707,0.077,5
5,1,18,Спорттовары,32410,743707,0.044,6
6,1,14,Рестораны,32328,743707,0.043,7
7,1,6,Транспорт,29041,743707,0.039,8
8,1,11,Красота,21380,743707,0.029,9
9,1,21,Цветы,19604,743707,0.026,10
